In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import random
import math

In [2]:
def show_img(img):
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    plt.show()

In [4]:
def circle_elem(radius):
    return cv.getStructuringElement(cv.MORPH_ELLIPSE, (2 * radius + 1, 2 * radius + 1), (radius, radius))

In [7]:
class Line:
    def __init__(self, contour):
        (cx, cy), (w, h), ang = cv.minAreaRect(contour)
        if w >= h:
            dx = (w / 2) * math.cos(ang * math.pi / 180)
            dy = (w / 2) * math.sin(ang * math.pi / 180)
        else:
            dx = (h / 2) * math.cos((ang - 90) * math.pi / 180)
            dy = (h / 2) * math.sin((ang - 90) * math.pi / 180)
            
        self.p1 = (int(cx + dx), int(cy + dy))
        self.p2 = (int(cx - dx), int(cy - dy))
        
    def cv_draw(self, target, color, thickness):
        cv.line(target, self.p1, self.p2, color, max(thickness, 1))

In [8]:
class Ellipse:
    def __init__(self, contour):
        self.ellipse = cv.fitEllipse(contour)
    
    def cv_draw(self, target, color, thickness):
        cv.ellipse(target, self.ellipse, color, thickness)

In [9]:
class ContourTree:
    def __init__(self, contour, children=None):
        self.contour = contour
        if children is not None and len(children) > 0:
            self.children = children
        else:
            self.children = None
            
    def __len__(self):
        return len(self.children) if self.children is not None else 0

    def __repr__(self):
        return f'<ContourTree children={self.children}>'

In [10]:
def extract_subtree(contours, hier, idx):
    children = []
    child = hier[idx][2] # First child
    while child >= 0:
        children.append(child)
        child = hier[child][0]
    
    child_trees = list(map(
        lambda c: extract_subtree(contours, hier, c), children))
    return ContourTree(contours[idx], child_trees)

In [11]:
def make_contour_tree(contours, hier):
    roots = []
    for i in range(len(hier)):
        if hier[i][3] < 0:
            # Top-level contour
            roots.append(extract_subtree(contours, hier, i))
            
    return roots

In [12]:
def is_line(contour, cutoff=0.6):
    a, b = cv.minAreaRect(contour)[1]
    try:
        ecc_mean = 2*a*b / (a*a + b*b)
        return ecc_mean < cutoff
    except:
        return False

In [13]:
def is_ellipse(contour, cutoff=0.5):
    try:
        (cx, cy), size, angle = cv.fitEllipse(contour)
    except:
        return False
    
    x_bb, y_bb, w_bb, h_bb = cv.boundingRect(contour)
    mask = np.zeros((h_bb, w_bb))
    thickness = int(max(w_bb / 60, 1))
    
    cv.drawContours(mask, [contour], 0, 255, thickness, offset=(-x_bb, -y_bb))
    orig_weight = np.sum(mask)
    
    cv.ellipse(mask, ((cx - x_bb, cy - y_bb), size, angle), 0, thickness * 4)
    masked_weight = np.sum(mask)
    
    return masked_weight / orig_weight < cutoff if orig_weight > 0 else False

In [15]:
def collect_lowest_shapes(shapes_depths):
    depth = 0
    all_shapes = []
    for shapes, d in shapes_depths:
        if d > depth:
            all_shapes = shapes.copy()
            depth = d
        elif d == depth:
            all_shapes.extend(shapes)
    return all_shapes, depth

In [16]:
def subtree_lowest_shapes(tree):
    """Finds the lowest-depth shapes in a contour tree.
    Return:
        Tuple of a list of shapes and the depth of the shapes.
        Note that the shapes may not come from the exact same subtree, but will
        all be at the same depth.
    """
    if len(tree) == 0:
        # Bottom-level tree
        if is_line(tree.contour):
            return [Line(tree.contour)], 0
        elif is_ellipse(tree.contour):
            try:
                return [Ellipse(tree.contour)], 0
            except:
                return [], -1
        else:
            print('Unknown shape')
            return [], -1
    else:
        def lowest_inc_depth(tree):
            s, d = subtree_lowest_shapes(tree)
            return s, d+1
        return collect_lowest_shapes(map(lowest_inc_depth, tree.children))

In [17]:
def all_lowest_shapes(roots):
    return collect_lowest_shapes(map(subtree_lowest_shapes, roots))[0]

In [18]:
def mask_out_shape(image, shape, thickness):
    mask = np.ones_like(image)
    shape.cv_draw(mask, 0, thickness)
    shape.cv_draw(mask, 0, -1) # TODO Does -thickness draw inside and outside?
    
    return image & mask

In [19]:
def extract_and_mask_lowest(image, mask_thick=None):
    if mask_thick is None:
        mask_thick = image.shape[0] // 15
        
    contours, hier = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    if contours is None or hier is None:
        return [], image
    
    contour_roots = make_contour_tree(contours, hier[0])
    shapes = all_lowest_shapes(contour_roots)
    
    mask = np.ones_like(image)
    for s in shapes:
        s.cv_draw(mask, 0, mask_thick)
        s.cv_draw(mask, 0, -1)
        
    masked = image & mask
    
    return shapes, masked

In [20]:
def draw_shapes(img_shape, shapes):
    thickness = img_shape[0] // 60
    target = np.zeros(img_shape)
    for s in shapes:
        s.cv_draw(target, 255, thickness)
        
    show_img(target)

In [21]:
def extract_recursive(image, mask_thick=None):
    all_shapes = []
    while True:
        shapes, new_image = extract_and_mask_lowest(image, mask_thick)
        image = new_image
        if len(shapes) > 0:
            all_shapes.extend(shapes)
        else:
            break
            
    return all_shapes

## Actually do stuff

In [ ]:
FILE_NAME = 'images/computer_drawn_simple.png'

In [47]:
orig_image = cv.imread(FILE_NAME, cv.IMREAD_GRAYSCALE)

In [48]:
processed = np.copy(orig_image)
cv.threshold(processed, 127, 255, cv.THRESH_BINARY_INV, processed)
dial_size = processed.shape[0] // 120
cv.dilate(processed, circle_elem(dial_size), processed)
#processed = cv.Canny(processed, 50, 150)
#cv.erode(processed, circle_elem(8), processed)

TypeError: Expected Ptr<cv::UMat> for argument '%s'

In [ ]:
show_img(orig_image)
show_img(processed)

In [ ]:
shapes = extract_recursive(processed)

In [ ]:
draw_shapes(processed.shape, shapes)